In [1]:
from pydantic_ai import (
    Agent,
    FunctionToolCallEvent,
    FunctionToolResultEvent,
    PartDeltaEvent,
    PartStartEvent,
    TextPartDelta,
)

from builder import AgentBuilder, AgentDep

In [8]:
agent = await AgentBuilder.build(
    name="WeatherAgent",
    user_id="1234",
    tool_names=["location", "weather"],
)

deps = AgentDep(
    user_name="id4thomas"
)

In [3]:
query="내 현재 위치와 주변 도시들을 알려주고 그 도시들의 날씨들도 알려줘."
# query="서울 주변 도시들을 알려주고 그 도시들의 날씨들도 알려줘. 도시 명칭은 반환값 그대로 사용해"
# query="도구를 사용해서 서울 날씨 알려줘. 도구의 결과도 json 그대로 알려줘"

# Run

In [ ]:
# ainvoke
result = await agent.run(
    query,
    deps=deps
)

In [5]:
print(result.output)

현재 위치와 주변 도시의 날씨 정보를 확인했습니다:

- **서울**: 1°C, 맑음, 습도 56%, 바람 12km/h  
- **성남**: 2°C, 구름 많음, 습도 57%, 바람 8km/h  
- **수원**: 2°C, 맑음, 습도 54%, 바람 9km/h  
- **창원**: 5°C, 구름 조금, 습도 56%, 바람 11km/h  

날씨 참고하시고, 좋은 하루 보내세요!


In [6]:
print(result.usage())

RunUsage(input_tokens=3791, output_tokens=370, requests=4, tool_calls=6)


# Stream

In [9]:
current_text = ""

async with agent.iter(query, deps=deps) as run:
    async for node in run:
        # print(type(node))
        if Agent.is_model_request_node(node):
            if Agent.is_model_request_node(node):
                print("[MODEL REQUEST NODE]")
            elif Agent.is_user_prompt_node(node):
                print("[USER REQUEST NODE]")
                
            async with node.stream(run.ctx) as stream:
                async for event in stream:
                    if isinstance(event, PartStartEvent):
                        # 첫 글자 여기 있음
                        if hasattr(event.part, 'content') and event.part.content:
                            current_text += event.part.content
                            print(event.part.content, end="", flush=True)

                    elif isinstance(event, PartDeltaEvent) and isinstance(event.delta, TextPartDelta):
                        delta = event.delta.content_delta
                        current_text += delta
                        print(delta, end="", flush=True)

        elif Agent.is_call_tools_node(node):
            print("[TOOL CALL NODE]")
            if current_text:
                print()
                current_text = ""

            async with node.stream(run.ctx) as stream:
                async for event in stream:
                    if isinstance(event, FunctionToolCallEvent):
                        print(f"\n[tool_call] tool={event.part.tool_name} args={event.part.args}")
                    elif isinstance(event, FunctionToolResultEvent):
                        print(f"[tool_result] {event.result.content}")

        elif Agent.is_end_node(node):
            print("[END NODE]")
            if current_text:
                print()
                current_text = ""
            print(f"{run.result.output}")

[MODEL REQUEST NODE]
알겠습니다. 현재 위치와 주변 도시 정보를 확인하고, 각 도시의 날씨를 확인해 드리겠습니다.
[TOOL CALL NODE]


[tool_call] tool=location_get_user_location args=None
[tool_result] {'name': 'Busan', 'coordinate': {'lat': 35.1796, 'lon': 129.0756}}
[MODEL REQUEST NODE]
현재 위치는 부산입니다. 부산 주변의 도시들을 확인해 보겠습니다.
[TOOL CALL NODE]


[tool_call] tool=location_get_nearby_cities args={"city_name": "Busan"}
[tool_result] [{'name': 'Gimhae', 'coordinate': {'lat': 35.2285, 'lon': 128.8894}}, {'name': 'Changwon', 'coordinate': {'lat': 35.228, 'lon': 128.6811}}, {'name': 'Ulsan', 'coordinate': {'lat': 35.5384, 'lon': 129.3114}}]
[MODEL REQUEST NODE]
부산 주변의 도시는 김해, 창원, 울산입니다. 각 도시의 날씨를 확인해 드리겠습니다.




[TOOL CALL NODE]


[tool_call] tool=weather_get_current_weather args={"city_name": "Busan"}

[tool_call] tool=weather_get_current_weather args={"city_name": "Gimhae"}

[tool_call] tool=weather_get_current_weather args={"city_name": "Changwon"}

[tool_call] tool=weather_get_current_weather args={"city_name": "Ulsan"}
[tool_resul